## Generating E-commerce product embeddings through a pre-trained BERT model using Product Description

In [1]:
import numpy as np 
import pandas as pd

import os 
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/products.csv
./data/order_products__prior.csv
./data/orders.csv
./data/aisles.csv
./data/departments.csv
./data/order_products__train.csv
./data/sample_submission.csv


In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd

from keras.utils import plot_model
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import time 
import pickle

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
from imblearn.over_sampling import SMOTE 
import warnings

from statsmodels.stats.outliers_influence import variance_inflation_factor 
import statsmodels.api as sm
from scipy.stats import kurtosis, skew
from scipy import stats
from scipy.stats import randint
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from math import sqrt
from sklearn.metrics import mean_squared_error
import xgboost
from sklearn.model_selection import RandomizedSearchCV
import shap
from keras import backend as K
# from keras.engine.topology import Layer
# from keras.utils.layer_utils import get_source_inputs
from keras import initializers, regularizers, constraints

from keras.layers import Dense, Input, LSTM, Bidirectional, Activation, Conv1D, GRU, TimeDistributed
from keras.layers import Dropout, Embedding, GlobalMaxPooling1D, MaxPooling1D, Add, Flatten, SpatialDropout1D
from keras.layers import GlobalAveragePooling1D, BatchNormalization, concatenate
from keras.layers import Reshape, Concatenate, Lambda, Average
# from keras.layers import Reshape, merge, Concatenate, Lambda, Average
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.initializers import Constant
# from keras.layers.merge import add
import tensorflow as tf
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence

from keras.utils import to_categorical
import transformers
import torch
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')
%matplotlib inline

2024-05-04 11:15:08.307143: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-04 11:15:08.872574: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/niranjan/miniconda3/envs/trainee/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df_products = pd.read_csv("./data/products.csv")
df_products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [4]:
df_products.shape

(49688, 4)

In [5]:
pretrained_weights = 'distilbert-base-uncased'
tokenizer = transformers.DistilBertTokenizer.from_pretrained(pretrained_weights)

In [6]:
tokenized = df_products.product_name.apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

In [7]:
dict(zip(tokenizer.all_special_tokens, tokenizer.all_special_ids))

{'[UNK]': 100, '[SEP]': 102, '[PAD]': 0, '[CLS]': 101, '[MASK]': 103}

In [8]:
text_len = [len(v) for v in tokenized]
print(text_len[:5])

[5, 6, 12, 14, 6]


In [9]:
print(tokenizer.encode("apple is my favourite food."))
print(tokenizer.encode("apple is"))
print(tokenizer.encode("eat eating ate eaten eats"))

[101, 6207, 2003, 2026, 8837, 2833, 1012, 102]
[101, 6207, 2003, 102]
[101, 4521, 5983, 8823, 8828, 20323, 102]


In [10]:
print(tokenized[0:2])

0        [101, 7967, 11642, 16324, 102]
1    [101, 2035, 1011, 3692, 5474, 102]
Name: product_name, dtype: object


In [11]:
maxlen = 20
for index, token in enumerate(tokenized):
    if len(token) >= maxlen:
        token = token[:maxlen]
    else:
        token = token + [0] * (maxlen - len(token))
    tokenized[index] = list(token)

In [12]:
tokenized = np.array(list(tokenized))

In [13]:
tokenized.shape

(49688, 20)

In [14]:
print(tokenized[0:2])

[[  101  7967 11642 16324   102     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [  101  2035  1011  3692  5474   102     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]]


In [15]:
bert_model = transformers.TFDistilBertModel.from_pretrained(pretrained_weights)
bert_model.summary()
plot_model(bert_model)

2024-05-04 11:15:31.291141: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-04 11:15:31.291398: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing

Model: "tf_distil_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
Total params: 66362880 (253.15 MB)
Trainable params: 66362880 (253.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


AttributeError: module 'pydot' has no attribute 'InvocationException'

In [16]:
product_ids = list(df_products.product_id)
product_name = list(df_products.product_name)

product_map = {}
for i in range(len(product_ids)):
    product_map[product_ids[i]] = product_name[i]

In [17]:
product_ids[:2], product_name[:2] 

([1, 2], ['Chocolate Sandwich Cookies', 'All-Seasons Salt'])

In [18]:
text_len[:5]

[5, 6, 12, 14, 6]

In [85]:
############################################################
### BREAK DOWN OF THE compress_tokens_embedding function ###
############################################################
tmp_arr = np.random.randn(80, 19, 768)
print("tmp_arr_shape: ", tmp_arr.shape)
tmp_i = 0
tmp_k = 400
tmp_x = np.mean(np.array(tmp_arr[tmp_i])[:text_len[tmp_k+tmp_i],:], axis = 0)
print("x shape: ", tmp_x.shape)

tmp_step = 12

output=[]
for j in range(0, tmp_arr.shape[2], tmp_step):
    output.append(np.mean(tmp_x[j:j+tmp_step]))

output=np.array(output)
print(output.shape)


tmp_arr_shape:  (80, 19, 768)
x shape:  (768,)
(64,)


In [86]:
#Will compress 768 bits embedding to 768/42 = 32 bits
def compress_tokens_embedding(arr,k,step):
    output = []
    r = arr.shape[0] # No of product descriptions
    c = arr.shape[2] # Embedding size
    for i in range(r):
        tmp = [product_ids[k+i],product_name[k+i]]
        # get the embedding of current product : 
        ### np.array(arr[i]) ==> shape: (19, 768)
        # get the embedding upto the text length : 
        ### np.array(arr[i])[:text_len[k+i], :] 
        # get the mean column wise
        x = np.mean(np.array(arr[i])[:text_len[k+i],:], axis = 0)
        for j in range(0,c,step): #Compression
            # the size of tmp will be (c // step + 2)
            # +2 from the initialization
            tmp.append(np.mean(x[j:j+step]))
        output.append(tmp)
    return np.array(output)

- Since we have 49.6K products & 49.6k * 768 dimension embedding becomes a lot to keep in memory, we process in batches.
- In first approach we will use the obtained hidden vector representation of all the tokens and do average pooling to get 768-dimension embedding ignoring the padded tokens.
- We will then compress the 768-dimension embedding to 64-dimensional by pooling across dimensions.


In [87]:
tokenized.shape

(49688, 20)

In [117]:
input_ids = tokenized
embedding_size = 768
step = 80
k = 0
avg_bits = 12
file_counter = 0
compressed_embedding_size = embedding_size//avg_bits # 768/12 = 64-dimensional embedding



In [118]:
########################################
### BREAK DOWN OF THE next cell code ###
########################################
print(tokenized.shape, type(tokenized))
tmp_k = 400
print("bert_model_embedding_shape: ", bert_model(tokenized[tmp_k:tmp_k + 80])[0].shape)
tmp_last_hidden_states = bert_model(tokenized[tmp_k:tmp_k + 80])[0][:, 1:maxlen + 1, :]
print("last_hidden_state_shape: ", tmp_last_hidden_states.shape)
output = compress_tokens_embedding(np.array(tmp_last_hidden_states), tmp_k, avg_bits)
print("output shape: ", output.shape)




(49688, 20) <class 'numpy.ndarray'>
bert_model_embedding_shape:  (80, 20, 768)
last_hidden_state_shape:  (80, 19, 768)
output shape:  (80, 66)


In [119]:
from tqdm import tqdm

In [121]:
# this cell code generates the csv file
# csv file contain k:k+step compressed embedding
# and product id and product name
with tqdm(total = input_ids.shape[0], desc="Compressing embedding") as pbar:
    while k < input_ids.shape[0]:
        last_hidden_states = bert_model(input_ids[k:k+step])[0][:,1:maxlen+1,:] #First token to 20th (max-len) token
        output = compress_tokens_embedding(np.array(last_hidden_states),k,avg_bits)
        col = ['product_id','product_name'] + [str(v) for v in range(compressed_embedding_size)]
        df_output = pd.DataFrame(output,columns = col)
        df_output.to_csv('embedding_data/file_' + str(file_counter) + '.csv', header = True, index=False)
        file_counter += 1
        k += step
        pbar.update(step)

Compressing embedding: 100%|█████████▉| 49680/49688 [05:15<00:00, 157.59it/s]


In [145]:
from annoy import AnnoyIndex

In [146]:
#####################################################################
##### BREAK DOWN OF create_index_find_similar_items() function#######
#####################################################################
dfs = []
files = 622 # total number of files
for i in range(files):
    dfs.append(pd.read_csv('embedding_data/file_'+ str(i)+'.csv'))
df_embedding = pd.concat(dfs)
print("df_embedding_shape", df_embedding.shape)
# df_embedding is the concatenation of all the embedding data csv file generated above
embedding_size = compressed_embedding_size
print("embedding size", embedding_size)

a = AnnoyIndex(embedding_size, 'euclidean')
for ix, row in df_embedding.iterrows():
    key = int(row['product_id'])
    vec = list(row[[str(v) for v in range(compressed_embedding_size)]])
    break

df_embedding_shape (49688, 66)
embedding size 64


In [150]:
#####################################################################
##### BREAK DOWN OF create_index_find_similar_items() function#######
#####################################################################
loaded_tree = AnnoyIndex(embedding_size,  'euclidean')
loaded_tree.load('test.tree')
top_k = 20

for ix,row in df_embedding.iterrows():
    item = int(row['product_id'])
    print(loaded_tree.get_nns_by_item(item, top_k))
    break

[1, 16143, 32469, 34370, 36307, 16613, 13820, 1624, 21100, 37836, 25398, 20949, 14869, 13764, 41793, 32422, 35633, 36278, 7983, 29905]


In [125]:
def create_index_find_similar_items(files):
    """
    files: file no.
    """
    # obtain all embeddings vector into a single file
    dfs = []
    for i in range(files):
        dfs.append(pd.read_csv('embedding_data/file_'+ str(i)+'.csv'))
    df_embedding = pd.concat(dfs)   
    dfs.append(pd.read_csv('embedding_data/file_'+ str(i)+'.csv'))
    embedding_size = compressed_embedding_size

    # add product key and vector to the AnnoyIndex
    a = AnnoyIndex(embedding_size, 'euclidean')
    for ix,row in df_embedding.iterrows():
        key = int(row['product_id'])
        vec = list(row[[str(v) for v in range(compressed_embedding_size)]])
        a.add_item(key,vec)
    
    a.build(100) # 100 trees
    a.save('test.tree')
    
    # load the same tree and file top 20
    u = AnnoyIndex(embedding_size,  'euclidean')
    u.load('test.tree')
    top_k = 20
    mat = []
    for ix,row in df_embedding.iterrows():
        item = int(row['product_id'])
        mat.append([item] + u.get_nns_by_item(item, top_k+1)[1:])
    
#     print(len(mat),len(mat[0]))
    
    cols = ['product_id']
    for i in range(top_k):
        cols += ['nearest_{}'.format(i+1)]
    print(cols)
    
    df_neighbors1 = pd.DataFrame(mat, columns = cols)

    df_neighbors2 = df_neighbors1.copy()

    # map from product id to product name
    for c in cols:
        df_neighbors2[c] = df_neighbors2[c].apply(lambda v : product_map[v])

    return df_neighbors2

In [126]:
df_neighbors = create_index_find_similar_items(files= 622)
df_neighbors.head(20)

['product_id', 'nearest_1', 'nearest_2', 'nearest_3', 'nearest_4', 'nearest_5', 'nearest_6', 'nearest_7', 'nearest_8', 'nearest_9', 'nearest_10', 'nearest_11', 'nearest_12', 'nearest_13', 'nearest_14', 'nearest_15', 'nearest_16', 'nearest_17', 'nearest_18', 'nearest_19', 'nearest_20']


,product_id,nearest_1,nearest_2,nearest_3,nearest_4,nearest_5,nearest_6,nearest_7,nearest_8,nearest_9,...,nearest_11,nearest_12,nearest_13,nearest_14,nearest_15,nearest_16,nearest_17,nearest_18,nearest_19,nearest_20
0,Chocolate Sandwich Cookies,Chocolate Chunk Cookies,Chocolate Butter Cookies,Chocolate Breakfast Bites,Milano Double Chocolate Cookies,Chocolate Chunk Cookie,Peanut Butter Cookies,Chocolate Cookies,Peanut Chocolate Candy,Milano Milk Chocolate Cookies,...,Vanilla Sandwich Cookies,Tray Cookies Chocolate Chip,Lemon Butter Cookies,Mini Chocolate Chunk Cookies,Lemon Twist Sandwich Cookies,Chocolate Peanut Butter,Chocolate Breakfast Biscuits Chocolate,Peanut Butter Breakfast Squares,Milano Orange Chocolate Cookies,Chocolate Chip Cookies
1,All-Seasons Salt,All-Bran Buds Cereal,Sour Gold-Bears,All-Purpose Flour,New Mama Bottom Spray,Season All Seasoned Salt,In-Wash Scent Booster,Light Thousand Island Dressing,Green Tea K-Cups,Safe-Cut Can Opener,...,Whole Bay Leaves,Whole Holland-Style onions,Eye Liner Black,Lip Shimmer - Fig,Cake Mate Decors Blue,Black Mission Figs,Soft-Picks,All American Salad,Supreme Matcha Green,Lip Shimmer - Plum
2,Robust Golden Unsweetened Oolong Tea,Unsweetened Golden Oolong Tea,Unsweetened Oolong Tea Shot,Original Iced Matcha Unsweetened Tea,White Tea Unsweetened Hint O'Mint,Unsweetened Organic Ginger Oasis Tea,Organic Unsweetened OriginalWhite Tea,Frozen Unsweetened Whole Strawberries,Unsweetened Pure & Smooth Green Tea,Organic Unsweetened Black Tea,...,Unsweetened Elegant Jasmine Green Tea,Unsweetened Lemongrass Green Tea,Unsweetened All Natural Black Coffee,Unsweetened Lemon Flavor Real Brewed Tea,Organic Unsweetened Berry Coconut Granola,Unsweetened Natural Big Cup Apple Sauce,Pure Leaf Unsweetened Real Brewed Tea,Unsweetened Mint Lemon Mate,Organic Unsweetened Lemon Iced Green Tea,Organic Unsweetened Creamy Cashewmilk
3,Smart Ones Classic Favorites Mini Rigatoni Wit...,Smart Ones Classic Favorites Thin Crust Pepper...,Sweet & Smoky Sloppy Joes Skillet Sauce,Chickpea & Tomato with Beef Stage 3 Baby Food,Hot Taco Lawry's Spices & Seasonings,Signature Ripe Tomato Extra Hot Gourmet Salsa,Grapefruit Bergamot Shea Butter Body Wash,Nutrish® Real Chicken & Veggies Super Premium ...,Smart Ones Tasty American Favorites Tuna Noodl...,Crunchy Oats 'n Honey Granola Bars,...,Crunchy Oats N' Honey Granola Bars,"Little Bites Seasonal Favorites, Pumpkin Muffins",Crunchy Oats 'n Dark Chocolate Granola Bars,Triscuit Whole Foods Brown Rice Tomato & Sweet...,Issimo Latte Macchiato Iced Coffee Drink with ...,Gluten Free Mini Carrot Cupcakes 12 ct,Naturally Rising Crust Canadian Style Bacon & ...,All Natural Chickpea Snacks! Smoky Chili & Lime,Danimals Strawberry Explosion Flavored Smoothie,Crunchy Organic Oats & Honey Granola Bars
4,Green Chile Anytime Sauce,Mushroom Italian Sauce,Honey Barbecue Sauce,Roasted Mole Sauce,Rustic Tomato Vodka Sauce,Alla Vodka Sauce,Light Creamy Alfredo Sauce,Mole Mexican Cooking Sauce,Hot Insanity Sauce,Butter Cooking Spray,...,Tomato Sauce Classic,Raw Carrot Soup,Southwest Chili Beans,Select Soy Sauce,Classic 1837 Steak Sauce,Sesame Cooking Sauce,Chile Verde Chicken,Tomato Basil Soup,Asian Chopped Salad,Green Curry Paste
5,Dry Nose Oil,Nasal Cleansing Pot,Dry Blackberry Soda,Ear Drying Drops,Walnut Spray Oil,Rich Chili Chips,Saline Nasal Mist,Perfect Pinch Vegetable Supreme,Cool Mint Gum,Scotch Packing Tape,...,Dry Erase Markers,Rose Essential Oil,Zero Dish Soap Fragrance Free,Rose Absolute Oil,Mild Pepper Rings,Lightly Dried Basil,Grapeseed Spray Oil,Zero Mixed Berry,Cherry Coke Zero,Buttery Spray
6,Pure Coconut Water With Orange,Pure Coconut Water With Pulp,Pure Coconut Water with Tropical Fruit,Coconut Water with Orange Mango,Pure Organic Coconut Water,Pure Coconut Water with Pineapple,Coconut Water with Pulp,Coconut Juice with Lime,"Coconut Water, Pure Organic",Organic Pure Extra Virgin Coconut Oil,...,Natural Pulp Free Coconut Water,Organic Coconut Water With Pineapple,"Coconut

In [127]:
df_neighbors.to_csv('neighbors_token_embedding_avg_compressed.csv', index=False, header=True)

In [128]:
df_neighbors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  49688 non-null  object
 1   nearest_1   49688 non-null  object
 2   nearest_2   49688 non-null  object
 3   nearest_3   49688 non-null  object
 4   nearest_4   49688 non-null  object
 5   nearest_5   49688 non-null  object
 6   nearest_6   49688 non-null  object
 7   nearest_7   49688 non-null  object
 8   nearest_8   49688 non-null  object
 9   nearest_9   49688 non-null  object
 10  nearest_10  49688 non-null  object
 11  nearest_11  49688 non-null  object
 12  nearest_12  49688 non-null  object
 13  nearest_13  49688 non-null  object
 14  nearest_14  49688 non-null  object
 15  nearest_15  49688 non-null  object
 16  nearest_16  49688 non-null  object
 17  nearest_17  49688 non-null  object
 18  nearest_18  49688 non-null  object
 19  nearest_19  49688 non-null  object
 20  neares